# Intra-factory Transport
## Capacitated Pickup and Delivery Problem with Time Windows

Factory automation allows companies to raise the quality and consistency of manufacturing processes while also allowing human workers to focus on safer, less repetitive tasks that have higher cognitive and creative demands.

In this scenario we have a set of intra-factory transport orders to move products at various stages in the assembly process from one processing station to another. Each station represents a particular type of manufacturing process and a given product may need to visit each processing station more than once. Multiple autonomous mobile robots (AMRs) with a fixed capacity will execute pickup and delivery orders between target locations, all with corresponding time_windows.

### Problem Details:
- 4 Locations each with an associated demand
    - 1 Start Location for AMRs

    - 3 Process Stations

- 3 AMRs with associated capacity

### Environment Setup
First, let's check if we have a GPU available in this environment.

In [ ]:
# Check for GPUs
!nvidia-smi

#### Install dependencies

In [20]:
# Install cuOpt

# Enable this in case you are running this in google colab or such places where cuOpt is not yet installed

# This would be incase underlying system is cuda-10.X 
# !pip install --user cuopt-cu11==25.5.* 

# This would be incase underlying system is cuda-12.x
# !pip install --user cuopt-cu12==25.5.

In [21]:
from cuopt import routing
from cuopt import distance_engine
import cudf
import numpy as np
import pandas as pd

- Hours of operation

In [22]:
factory_open_time = 0
factory_close_time = 100

![waypoint_graph.png not found](waypoint_graph.png "Waypoint Graph")

### Waypoint Graph

#### Compressed Sparse Row (CSR) representation of above weighted waypoint graph.
For details on the CSR encoding of the above graph see the [cost_matrix_and_waypoint_graph_creation.ipynb](cost_matrix_and_waypoint_graph_creation.ipynb) notebook.

In [23]:
offsets = np.array([0, 1, 3, 7, 9, 11, 13, 15, 17, 20, 22])
edges =   np.array([2, 2, 4, 0, 1, 3, 5, 2, 6, 1, 7, 2, 8, 3, 9, 4, 8, 5, 7, 9, 6, 8])
weights = np.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2, 2, 1, 2])

#### Select specific waypoints in the graph as target locations
In this case we would like the AMRs to begin from waypoint 0 and service locations 4, 5, and 6.

In [24]:
target_locations = np.array([0, 4, 5, 6])

### Cost Matrix

#### Use cuOpt to calculate the corresponding cost matrix
Here we will be using a single cost matrix representing time.

In [ ]:
waypoint_graph = distance_engine.WaypointMatrix(
    offsets,
    edges,
    weights
)
time_matrix = waypoint_graph.compute_cost_matrix(target_locations)
target_map = {v:k for k, v in enumerate(target_locations)}
index_map = {k:v for k, v in enumerate(target_locations)}
print(f"Waypoint graph node to time matrix index mapping \n{target_map}\n")
print(time_matrix)

### Transport Orders

Setup Transport Order Data

The transport orders dictate the movement of parts from one area of the factory to another.  In this example nodes 4, 5, and 6 represent the processing stations that parts must travel between and deliveries to node 0 represent the movement of parts off the factory floor.

In [ ]:
transport_order_data = cudf.DataFrame({
    "pickup_location":       [4,  5,  6,  6,  5,  4],
    "delivery_location":     [5,  6,  0,  5,  4,  0],
    "order_demand":          [1,  1,  1,  1,  1,  1],
    "earliest_pickup":       [0,  0,  0,  0,  0,  0],
    "latest_pickup":         [10, 20, 30, 10, 20, 30],
    "pickup_service_time":   [2,  2,  2,  2,  2,  2],
    "earliest_delivery":     [0,  0,  0,  0,  0,  0],
    "latest_delivery":       [45, 45, 45, 45, 45, 45],
    "delivery_serivice_time":[2,  2,  2,  2,  2,  2]
})
transport_order_data

### AMR Data

Set up AMR fleet data

In [ ]:
n_robots = 2
robot_data = {
    "robot_ids": [i for i in range(n_robots)],
    "carrying_capacity":[2, 2]
}
robot_data = cudf.DataFrame(robot_data).set_index('robot_ids')
robot_data

### cuOpt DataModel View

Setup the routing.DataModel.

In [28]:
n_locations = len(time_matrix)
n_vehicles = len(robot_data)

# a pickup order and a delivery order are distinct with additional pad for the depot with 0 demand
n_orders = len(transport_order_data) * 2
data_model = routing.DataModel(n_locations, n_vehicles, n_orders)
data_model.add_cost_matrix(time_matrix)


#### Set the per order demand

From the perspective of the cuOpt solver_settings, each distinct transaction (pickup order or delivery order) are treated separately with demand for pickup denoted as positive and the corresponding delivery treated as negative demand.

In [ ]:
# This is the number of parts that needs to be moved
raw_demand = transport_order_data["order_demand"]

# When dropping off parts we want to remove one unit of demand from the robot
drop_off_demand = raw_demand * -1

# Create pickup and delivery demand
order_demand = cudf.concat([raw_demand, drop_off_demand], ignore_index=True)

order_demand

In [30]:
# add the capacity dimension
data_model.add_capacity_dimension("demand", order_demand, robot_data['carrying_capacity'])

#### Setting Order locations

set the order locations and pickup and delivery pairs.

In [ ]:
pickup_order_locations = cudf.Series([target_map[loc] for loc in transport_order_data['pickup_location'].to_arrow().to_pylist()])
delivery_order_locations = cudf.Series([target_map[loc] for loc in transport_order_data['delivery_location'].to_arrow().to_pylist()])
order_locations = cudf.concat([pickup_order_locations, delivery_order_locations], ignore_index=True)

print(order_locations)

# add order locations
data_model.set_order_locations(order_locations)

#### Mapping pickups to deliveries

In [32]:
# IMPORTANT NOTE : pickup and delivery pairs are indexed into the order locations array.
npair_orders = int(len(order_locations)/2)
pickup_orders = cudf.Series([i for i in range(npair_orders)])
delivery_orders = cudf.Series([i + npair_orders for i in range(npair_orders)])
# add pickup and delivery pairs.
data_model.set_pickup_delivery_pairs(pickup_orders, delivery_orders)

#### Time Windows

In [33]:
# create earliest times
vehicle_earliest_time = cudf.Series([factory_open_time] * n_vehicles)
order_time_window_earliest = cudf.concat([transport_order_data["earliest_pickup"], transport_order_data["earliest_delivery"]], ignore_index=True)

# create latest times
vehicle_latest_time = cudf.Series([factory_close_time] * n_vehicles)
order_time_window_latest = cudf.concat([transport_order_data["latest_pickup"], transport_order_data["latest_delivery"]], ignore_index=True)

# create service times
order_service_time = cudf.concat([transport_order_data["pickup_service_time"], transport_order_data["delivery_serivice_time"]], ignore_index=True)

# add time window constraints
data_model.set_order_time_windows(order_time_window_earliest, order_time_window_latest)
data_model.set_order_service_times(order_service_time)
data_model.set_vehicle_time_windows(vehicle_earliest_time, vehicle_latest_time)

### CuOpt SolverSettings

Set up routing.SolverSettings.

In [34]:
solver_settings = routing.SolverSettings()

# solver_settings will run for given time limit.  Larger and/or more complex problems may require more time.
solver_settings.set_time_limit(5)

### Solution

In [ ]:
routing_solution = routing.Solve(data_model, solver_settings)
if routing_solution.get_status() == 0:
    print("Cost for the routing in time: ", routing_solution.get_total_objective())
    print("Vehicle count to complete routing: ", routing_solution.get_vehicle_count())
    print(routing_solution.route)
else:
    print("NVIDIA cuOpt Failed to find a solution with status : ", routing_solution.get_status())

#### Converting solution to waypoint graph

Because we maintained the mapping between cost matrix indices and locations in the waypoint graph we can now convert our solution to reference the nodes in the waypoint graph corresponding to the selected target locations.

In [ ]:
target_loc_route = [index_map[loc] for loc in routing_solution.route['location'].to_arrow().to_pylist()]
routing_solution.route['order_array_index'] = routing_solution.route['route']
routing_solution.route['route'] = target_loc_route
print(routing_solution.route)

#### Convert routes from target location based routes to waypoint level routes

In [ ]:
unique_robot_ids = routing_solution.route['truck_id'].unique()
all_routes = routing_solution.get_route()

for robot in unique_robot_ids.to_arrow().to_pylist():
    route = all_routes[all_routes['truck_id']==robot]
    waypoint_route = waypoint_graph.compute_waypoint_sequence(target_locations, route)
    print(f"Target location level route for robot {robot}:\n{all_routes[all_routes['truck_id']==robot]['route']}\n\n")
    print(f"Waypoint level route for robot {robot}:\n{waypoint_route}\n\n")


SPDX-FileCopyrightText: Copyright (c) 2025 NVIDIA CORPORATION & AFFILIATES. All rights reserved.
SPDX-License-Identifier: MIT
Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the "Software"), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions: The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.